In [136]:
# import lib
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout
from keras.regularizers import L1L2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, auc

sys.path.append("../scripts")
from data_loader import get_stocks, get_etfs, get_technical_indicators

In [12]:
# define global constant
STOCKS = ['AAPL', 'MSFT', 'GOOGL']
HORIZONS = [1, 3, 7, 30, 90] # 1 day, 3 days, 1 week, 1 month, 3 months 

In [152]:
def prepare_data(stock, testing_size, seq_length=720, horizon=1, threshold=0.01): 
    # load the corresponding stock
    df = get_stocks(stock)
    df = get_technical_indicators(df)
    df = df[len(df)-testing_size:]

    # set up the data 
    df['Price_Change'] = (df['Close'].shift(-horizon) - df['Close']) / df['Close']
    df['Target'] = (df['Price_Change'] > threshold).astype(int) | (df['Price_Change'] < -threshold).astype(int)
    X = df.drop(columns=['Date', 'Stock', 'Target', 'Price_Change']) # drop the irrevalent variables for training set
    y = df['Target'].values

    # normalise the vector
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    
    return X, y, scaler

In [90]:
def get_sequences(X, y, seq_length):
    """Create properly aligned sequences where:
    - Each X sequence contains seq_length past observations
    - Each y sequence contains corresponding targets
    - Both have exactly the same length"""
    X_seq, y_seq = [], []
    for i in range(seq_length, len(X)):
            X_seq.append(X[i-seq_length:i])  # Past seq_length observations
            y_seq.append(y[i])   # Corresponding targets
    return np.array(X_seq), np.array(y_seq)

In [154]:
# build lstm model
def get_enhanced_lstm(seq_len, num_features):
    model = Sequential([
        Input(shape=(seq_len, num_features)),
        LSTM(128, return_sequences=True, 
             kernel_regularizer=L1L2(l1=1e-5, l2=1e-4),
             recurrent_dropout=0.2),
        BatchNormalization(),
        LSTM(64, return_sequences=True,
             kernel_regularizer=L1L2(l1=1e-5, l2=1e-4)),
        Dropout(0.3),
        LSTM(32),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    optimizer = Adam(learning_rate=0.001)
    # model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# model.summary()

In [132]:
def evaluate_classification(y_true, y_pred_probs, threshold=0.5):
    """
    Compute comprehensive classification metrics
    Args:
        y_true: True binary labels (0 or 1)
        y_pred_probs: Predicted probabilities (0 to 1)
        threshold: Cutoff for binary classification
    Returns:
        Dictionary of metrics and plots
    """
    # Convert probabilities to binary predictions
    y_pred = (y_pred_probs >= threshold).astype(int)
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_probs),
    }
    
    return metrics

In [168]:
# train the model by rolling window and predict the outcome
def rolling_window_train(stock, horizon=1, train_ratio=0.8, window_size=720, epochs=5, batch_size=32, testing_size=1000):
    predictions = []
    actuals = []
    probability_predictions = []
    
    # prepare data for the seq_length
    X, y, scaler = prepare_data(stock, seq_length=window_size, testing_size=testing_size, horizon=horizon)
    test_start = int(len(X)*train_ratio)
    
    # get sequence data
    X, y = get_sequences(X, y, window_size)

    i = test_start
    while (i < len(X)):
        i_seq = i-window_size
        # reset the model
        model = get_enhanced_lstm(window_size, X.shape[2])

        # set up the variables for training
        X_window, y_window = X[i_seq-window_size:i_seq], y[i_seq-window_size:i_seq]
        steps = window_size // batch_size // epochs

        # fit the model with rolling window
        model.fit(X_window, y_window, epochs=epochs, batch_size=batch_size, steps_per_epoch=steps, verbose=0)

        # Predict next value
        pred = model.predict(X[i_seq:i_seq+1])
        binary_pred = 1 if pred > 0.5 else 0

        predictions.append(binary_pred)
        actuals.append(y[i_seq])
        probability_predictions.append(pred)

        i += 1

    return np.array(predictions).flatten(), np.array(actuals).flatten(), np.array(probability_predictions).flatten()

In [146]:
def expanding_window_train(stock, horizon, train_ratio=0.8, window_size=720, epochs=5, batch_size=32, testing_size=1000):
    predictions = []
    actuals = []
    probability_predictions = []
    X, y, scaler = prepare_data(stock, seq_length=window_size, testing_size=testing_size, horizon=horizon)
    test_start = int(len(X)*train_ratio)

    i = test_start
    while (i < len(X)):
        i_seq = i-window_size
        X_seq, y_seq = get_sequences(X, y, window_size)
        # reset the model
        model = get_enhanced_lstm(window_size, X_seq.shape[2])

        # set up the variables for training
        X_window, y_window = X_seq[i_seq-window_size:i_seq], y_seq[i_seq-window_size:i_seq]
        steps = window_size // batch_size // epochs

        # fit the model with rolling window
        model.fit(X_window, y_window, epochs=epochs, batch_size=batch_size, steps_per_epoch=steps, verbose=0)

        # Predict next value
        # print(X_seq[i_seq:i_seq+1].shape)
        # pred = model.predict(X_seq[i_seq:i_seq+1].reshape(1, window_size, X_seq.shape[2]))
        # pred = scaler.inverse_transform(pred[0,0].reshape(-1,1))
        # predictions.append(pred)
        # y_test_original = scaler.inverse_transform(y_seq[i_seq][0].reshape(-1,1))
        # actuals.append(y_test_original)

        pred = model.predict(X_seq[i_seq:i_seq+1])
        binary_pred = 1 if pred > 0.5 else 0

        predictions.append(binary_pred)
        actuals.append(y_seq[i_seq])
        probability_predictions.append(pred)

        i += 1
        window_size += 1

    return np.array(predictions).flatten(), np.array(actuals).flatten(), np.array(probability_predictions).flatten()

In [172]:
def main():
    print("training & predicting by roll window:\n")
    roll_predictions, roll_actuals, roll_probability_predictions = rolling_window_train(STOCKS[0], HORIZONS[0], window_size=10, epochs=2, batch_size=5, testing_size=100)
    print("training & predicting by expand window\n")
    expand_predictions, expand_actuals, expand_probability_predictions = expanding_window_train(STOCKS[0], HORIZONS[0], window_size=10, epochs=2, batch_size=5, testing_size=100)
    
    # evaulate the model performance
    roll_matrics = evaluate_classification(roll_actuals, roll_probability_predictions, threshold=0.5)
    expand_matrics = evaluate_classification(expand_actuals, expand_probability_predictions, threshold=0.5)

    print(f"roll_matrics:\n{roll_matrics}\n", f"expand_matrics:\n{expand_matrics}")

In [174]:
if __name__ == '__main__':
    main()

training & predicting by roll window:

✅ Processed data already exists. Skipping processing.
✅ Technical indicators added successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
training & predicting by expand window

✅ Processed data already exists. Skipping processing.
✅ Technical indicators added successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 